# Regresión

Los objetivos de esta clase son:

* Comprender/recordar regresión lineal.
* Estimar el error al aplicar modelos matemáticos a los datos.
* Introducir la librería _scikit-learn_.
* Otros tipos de regresión.

## Motivación

La regresión lineal es una técnica universalmente utilizada y a pesar de su simpletaza, la derivación de este método entrega importantes consideraciones sobre su implementación, sus hipótesis y sus posibles extensiones.

Para motivar el estudio utilizaremos datos de diabetes disponibles en la biblioteca _scikit learn_.

In [1]:
import numpy as np
import pandas as pd
import altair as alt

from sklearn import datasets

alt.themes.enable('opaque')  # Para quienes utilizan temas oscuros en Jupyter Lab

ThemeRegistry.enable('opaque')

In [2]:
diabetes_X, diabetes_y = datasets.load_diabetes(return_X_y=True, as_frame=True)
diabetes = pd.concat([diabetes_X, diabetes_y], axis=1)
diabetes.head()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019908,-0.017646,151.0
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068330,-0.092204,75.0
2,0.085299,0.050680,0.044451,-0.005671,-0.045599,-0.034194,-0.032356,-0.002592,0.002864,-0.025930,141.0
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022692,-0.009362,206.0
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031991,-0.046641,135.0


En este conjunto la variable a predecir (_target_) es una medida cuantitativa de la progresión de la enfermedad un año después de la línea base. Contiene 10 características (_features_) numéricas, definidas en la siguienta tabla. Cada una de estas ha sido centrada y escalada por su desviación estandar multiplicada por la cantidad de muestras, i.e. la suma de cuadrados de cada columna es 1, que en la prácitca es que tengan norma unitaria.

| Feature       | Descripción     |
| :------------- | :----------: |
| age | age in years|
| sex | sex |
| bmi | body mass index|
| bp | average blood pressure|
| s1 | tc, T-Cells (a type of white blood cells)|
| s2 | ldl, low-density lipoproteins|
| s3 | hdl, high-density lipoproteins|
| s4 | tch, thyroid stimulating hormone|
| s5 | ltg, lamotrigine|
| s6 | glu, blood sugar level|

A continuación exploremos los datos

In [3]:
diabetes.describe().T

,count,mean,std,min,25%,50%,75%,max
age,442.0,-3.634285e-16,0.047619,-0.107226,-0.037299,0.005383,0.038076,0.110727
sex,442.0,1.308343e-16,0.047619,-0.044642,-0.044642,-0.044642,0.050680,0.050680
bmi,442.0,-8.045349e-16,0.047619,-0.090275,-0.034229,-0.007284,0.031248,0.170555
bp,442.0,1.281655e-16,0.047619,-0.112400,-0.036656,-0.005671,0.035644,0.132044
s1,442.0,-8.835316e-17,0.047619,-0.126781,-0.034248,-0.004321,0.028358,0.153914
s2,442.0,1.327024e-16,0.047619,-0.115613,-0.030358,-0.003819,0.029844,0.198788
s3,442.0,-4.574646e-16,0.047619,-0.102307,-0.035117,-0.006584,0.029312,0.181179
s4,442.0,3.777301e-16,0.047619,-0.076395,-0.039493,-0.002592,0.034309,0.185234
s5,442.0,-3.830854e-16,0.047619,-0.126097,-0.033249,-0.001948,0.032433,0.133599
s6,442.0,-3.412882e-16,0.047619,-0.137767,-0.033179,-0.001078,0.027917,0.135612


In [4]:
diabetes.apply(np.linalg.norm)

age          1.000000
sex          1.000000
bmi          1.000000
bp           1.000000
s1           1.000000
s2           1.000000
s3           1.000000
s4           1.000000
s5           1.000000
s6           1.000000
target    3584.818126
dtype: float64

También es interesante ver como se relaciona cada _feature_ con el _target_.

In [5]:
base = alt.Chart(diabetes).mark_circle().encode(
    x=alt.X(alt.repeat("row"), type='quantitative'),
    y="target"
).properties(
    width=300,
    height=300
)

base.repeat(row=diabetes_X.columns.tolist()[:4]) | base.repeat(row=diabetes_X.columns.tolist()[4:7]) | base.repeat(row=diabetes_X.columns.tolist()[7:])

alt.HConcatChart(...)

## Modelo

Supondremos que tenemos $m$ datos. 

Cada dato $x^{(i)}$, $i=1,\dots,$ $m$ tiene $n$ componentes,
$x^{(i)} = (x^{(i)}_1, ..., x^{(i)}_n)$. 

Conocemos además el valor (etiqueta) asociado a $x^{(i)}$ que llamaremos $y^{(i)}$, $i=1,\dots, m$ .

Nuestra hipótesis de modelo lineal puede escribirse como

$$\begin{aligned}
h_{\theta}(x) &= \theta_0 + \theta_1 x_1 + \theta_2 x_2 + ... + \theta_n x_n \\
          &= \begin{bmatrix}\theta_0 & \theta_1 & \theta_2 & \dots & \theta_n\end{bmatrix} \begin{bmatrix}1 \\ x_1 \\x_2 \\ \vdots \\ x_n\end{bmatrix} \\
          &= \theta^T \begin{bmatrix}1\\x\end{bmatrix} = \begin{bmatrix}1 & x^T\end{bmatrix} \theta \end{aligned}$$

Definiremos $x^{(i)}_0 =1$, de modo que
$h_{\theta}(x^{(i)}) = (x^{(i)})^T \theta $ y buscamos el vector de parámetros
$$\theta = \begin{bmatrix}\theta_0 \\ \theta_1 \\ \theta_2 \\ \vdots \\ \theta_n\end{bmatrix}$$


Definamos las matrices

$$\begin{aligned}
Y &= \begin{bmatrix}y^{(1)} \\ y^{(2)} \\ \vdots \\ y^{(m)}\end{bmatrix}\end{aligned}$$

y

$$\begin{aligned}
X = 
\begin{bmatrix} 
1 & x^{(1)}_1 & \dots & x^{(1)}_n \\ 
1 & x^{(2)}_1 & \dots & x^{(2)}_n \\
\vdots & \vdots & & \vdots \\
1 & x^{(m)}_1 & \dots & x^{(m)}_n \\
\end{bmatrix}
= 
\begin{bmatrix} 
- (x^{(1)})^T - \\ 
- (x^{(2)})^T - \\
\vdots \\
- (x^{(m)})^T - \\
\end{bmatrix}\end{aligned}$$


Luego la evaluación
de todos los datos puede escribirse matricialmente como

$$\begin{aligned}
X \theta &= 
\begin{bmatrix}
1 & x_1^{(1)} & ... & x_n^{(1)} \\
\vdots & \vdots & & \vdots \\
1 & x_1^{(m)} & ... & x_n^{(m)} \\
\end{bmatrix}
\begin{bmatrix}\theta_0 \\ \theta_1 \\ \vdots \\ \theta_n\end{bmatrix} \\
& = 
\begin{bmatrix}
1 \theta_0 + x^{(1)}_1 \theta_1 + ... + x^{(1)}_n \theta_n \\
\vdots \\
1 \theta_0 + x^{(m)}_1 \theta_1 + ... + x^{(m)}_n \theta_n \\
\end{bmatrix} \\
& = 
\begin{bmatrix}
h(x^{(1)}) \\
\vdots \\
h(x^{(m)})
\end{bmatrix}\end{aligned}$$


Nuestro problema es
encontrar un “buen” conjunto de valores $\theta$ de modo que

$$\begin{aligned}
\begin{bmatrix}
h(x^{(1)}) \\
h(x^{(2)}) \\
\vdots \\
h(x^{(m)})
\end{bmatrix}
\approx
\begin{bmatrix}y^{(1)} \\ y^{(2)} \\ \vdots \\ y^{(m)}\end{bmatrix}\end{aligned}$$

es decir, que $$X \theta \approx Y$$

Para encontrar el mejor vector $\theta$ podríamos definir una función de costo $J(\theta)$ de la siguiente manera:

$$J(\theta) = \frac{1}{2} \sum_{i=1}^{m} \left( h_{\theta}(x^{(i)}) - y^{(i)}\right)^2$$

## Implementaciones

### Aproximación Ingenieril

¿Cómo podemos resolver el problema
en el menor número de pasos?

Deseamos resolver el sistema $$A \theta = b$$ con
$A \in \mathbb{R}^{m \times n}$ y $m > n$ (La matrix $A$ es skinny).

¿Cómo resolvemos?

Bueno,
si $A \in \mathbb{R}^{m \times n}$, entonces
$A^T \in \mathbb{R}^{n \times m}$ y la multiplicación está bien definida
y obtengo el siguiente sistema lineal, conocido como **Ecuación Normal**:
$n \times n$.
$$(A^T A) \  \theta = A^T b$$ 

Si la matriz $A^T A$ es invertible, el sistema se puede solucionar “sin mayor reparo”. $$\theta = (A^T A)^{-1} A^T b$$

En nuestro caso, obtendríamos $$\theta = (X^T X)^{-1} X^T Y$$ Esta
respuesta, aunque correcta, no admite interpretaciones y no permite
generalizar a otros casos más generales.

En particular...

-   ¿Qué relación tiene con la función de costo (no) utilizada?

-   ¿Qué pasa si $A^T A$ no es invertible?


### Aproximación Machine Learning

¿Cómo podemos obtener una
buena aproximación para $\theta$?

Queremos encontrar $\theta^*$ que minimice $J(\theta)$.

Basta con utilizar una buena rutina de optimización para cumplir con
dicho objetivo.

En particular, una elección natural es tomar la dirección de mayor
descenso, es decir, el método del máximo descenso (gradient descent).

$$\theta^{(n+1)} = \theta^{(n)} - \alpha \nabla_{\theta} J(\theta^{(n)})$$
donde $\alpha >0$ es la tasa de aprendizaje.

En
nuestro caso, puesto que tenemos
$$J(\theta) = \frac{1}{2} \sum_{i=1}^{m} \left( h_{\theta}(x^{(i)}) - y^{(i)}\right)^2$$
se tiene que

$$\begin{aligned}
\frac{\partial J(\theta)}{\partial \theta_k} &=
\frac{\partial }{\partial \theta_k} \frac{1}{2} \sum_{i=1}^{m} \left( h_{\theta}(x^{(i)}) - y^{(i)}\right)^2 \\
&= \frac{1}{2} \sum_{i=1}^{m}  2 \left( h_{\theta}(x^{(i)}) - y^{(i)}\right) \frac{\partial h_{\theta}(x^{(i)})}{\partial \theta_k}  \\
&= \sum_{i=1}^{m} \left( h_{\theta}(x^{(i)}) - y^{(i)}\right) x^{(i)}_k\end{aligned}$$


Luego, el algoritmo queda como sigue:
$$\begin{aligned}
\theta^{(n+1)} & = \theta^{(n)} - \alpha \nabla_{\theta} J(\theta^{(n)}) \\\\
\frac{\partial J(\theta)}{\partial \theta_k}
&= \sum_{i=1}^{m} \left( h_{\theta}(x^{(i)}) - y^{(i)}\right) x^{(i)}_k\end{aligned}$$

**Observación**: La elección de $\alpha$ es crucial para la convergencia. En
particular, una regla de trabajo es utilizar $0.01/m$. Notar que el parámetro $\alpha$ no es un parámetro del modelo como tal, si no que es parte del algoritmo, este tipo de parámetros se suelen llamar **hyperparameters**. Pudes reconocerlos porque el valor del parámetro es conocido antes de la fase de entrenamiento del modelo.


In [6]:
def lms_regression_slow(X, Y, theta, tol=1E-6):
    m, n = X.shape
    converged = False
    alpha = 0.01 / len(Y)
    while not converged:
        gradient = 0.
        for xiT, yi in zip(X, Y):
            xiT = xiT.reshape(1, n)
            hi = np.dot(xiT, theta)
            gradient += (hi - yi) * xiT.T
        new_theta = theta - alpha * gradient
        converged = np.linalg.norm(theta - new_theta) < tol * np.linalg.norm(theta)
        theta = new_theta
    return theta

In [7]:
m = 1000
t = np.linspace(0, 1, m)
x = 2 + 2 * t
y = 300 + 100 * t
X = np.array([np.ones(m), x]).T
Y = y.reshape(m, 1)
theta_0 = np.array([[0.0], [0.0]])

In [8]:
theta_slow = lms_regression_slow(X, Y, theta_0)
print(theta_slow)

[[199.39672176]
 [ 50.19457286]]


Validamos si nuestro resultado es el indicado con una tolerancia dada.

In [9]:
np.allclose(X @ theta_slow, Y, atol=0.5)

True

In [10]:
np.allclose(X @ theta_slow, Y, atol=1e-3)

False

### Implementación Vectorial

**¿Cómo podemos obtener una justificación para la ecuación normal?**

Necesitamos los siguientes ingredientes:

$$\begin{aligned}
\nabla_x &(x^T A x) = A x + A^T x \\ 
\nabla_x &(b^T x) = b \end{aligned}$$

Se tiene

$$\begin{aligned}
J(\theta) 
&= \frac{1}{2} \sum_{i=1}^{m} \left( h_{\theta}(x^{(i)}) - y^{(i)}\right)^2 \\
&= \frac{1}{2} \sum_{i=1}^{m} \left( h_{\theta}(x^{(i)}) - y^{(i)}\right) \left( h_{\theta}(x^{(i)}) - y^{(i)}\right) \\
&= \frac{1}{2} \left( X \theta - Y \right)^T \left( X \theta - Y \right) \\
&= \frac{1}{2} \left( \theta^T X^T - Y^T \right) \left( X \theta - Y \right) \\
&= \frac{1}{2} \left( \theta^T X^T X \theta - \theta^T X^T Y - Y^T X \theta + Y^T Y \right) \\
&= \frac{1}{2} \left( \theta^T X^T X \theta - 2 (Y^T X) \theta + Y^T Y \right)\end{aligned}$$

Aplicando a cada uno de los términos, obtenemos:

$$\begin{aligned}
\nabla_\theta ( \theta^T X^T X \theta ) &= X^T X \theta + (X^T X)^T \theta \\
& = 2 X^T X \theta\end{aligned}$$

también se tiene

$$\begin{aligned}
\nabla_\theta ( Y^T X \theta  ) &= (Y^T X) ^T\\
&= X^T Y\end{aligned}$$

y por último

$$\begin{aligned}
\nabla_\theta ( Y^T Y  ) = 0\end{aligned}$$

Por lo tanto se tiene que

$$\begin{aligned}
\nabla_\theta J(\theta) 
& = \nabla_\theta  \frac{1}{2} \left( \theta^T X^T X \theta - 2 (Y^T X) \theta + Y^T Y \right) \\
&= \frac{1}{2} ( 2 X^T X \theta - 2 X^T Y + 0 ) \\
&= X^T X \theta - X^T Y \end{aligned}$$


Esto significa que el problema $$\min_\theta J(\theta)$$ se resuelve al
hacer todas las derivadas parciales iguales a cero (ie, gradiente igual
a cero) $$\nabla_\theta J(\theta) = 0$$ lo cual en nuestro caso se
convierte convenientemente a la ecuación normal $$X^T X \theta = X^T Y$$
y se tiene $$\hat{\theta} = (X^T X)^{-1} X^T Y$$
Aquí $\hat{\theta}$ es una estimación de $\theta$. 

In [11]:
def lms_regression_fast(X, Y, theta, tol=1E-6):
    converged = False
    alpha = 0.01 / len(Y)
    theta = theta.reshape(X.shape[1], 1)
    A = np.dot(X.T, X)
    b = np.dot(X.T, Y)
    while not converged:
        gradient = np.dot(A, theta) - b
        new_theta = theta - alpha * gradient
        converged = np.linalg.norm(theta - new_theta) < tol * np.linalg.norm(theta)
        theta = new_theta
    return theta

In [12]:
theta_fast = lms_regression_fast(X, Y, theta_0)
print(theta_fast)

[[199.39672176]
 [ 50.19457286]]


Validación

In [13]:
np.allclose(X @ theta_fast, Y, atol=0.5)

True

In [14]:
np.allclose(X @ theta_fast, Y, atol=1e-3)

False

También es posible usar la implementación de resolución de sistemas lineales dispoinible en numpy.

In [15]:
def matrix_regression(X, Y, theta, tol=1E-6):
    A = np.dot(X.T,X)
    b = np.dot(X.T,Y)
    sol = np.linalg.solve(A,b)
    return sol

In [16]:
theta_npsolve = matrix_regression(X, Y, theta_0)
print(theta_npsolve)

[[200.]
 [ 50.]]


### Interpretación Probabilística

Consideremos el modelo lineal

$$ Y = X \theta + \varepsilon $$

donde $\varepsilon$ es un vector de errores aleatorios de media cero y matriz de dispersión $\sigma^2 I$, donde $I$ es la matriz identidad. Es usual añadir el supuesto de normalidad al vector de errores, por lo que se asume que 

$$\varepsilon \sim  \mathcal{N}(0, \sigma^2 I)$$

Cabe destacar que:

-   $\theta$ no es una variable aleatoria, es un parámetro
    (desconocido).
-   $Y \ | \ X; \theta \sim \mathcal{N}(X \theta, \sigma^2 I)$


La función de verosimilitud $L(\theta)$ nos
permite entender que tan probable es encontrar los datos observados,
para una elección del parámetro $\theta$.

$$
L(\theta) = \left( 2 \pi \sigma^2 \right)^{-n/2} \, \exp\left(- \frac{1}{2 \sigma ^2} || Y - X \theta ||^2 \right)
$$

Sea $l(\theta) = \log L(\theta)$ la log-verosimilitud. Luego, ignorando los términos constantes se tiene

$$
l(\theta) = -\frac{n}{2} \log \sigma^2 - \frac{1}{2 \sigma ^2} || Y - X \theta ||^2
$$

Luego, derivando respecto a $\theta$:

$$
\begin{aligned}
\frac{\partial l(\theta)}{\partial \theta}
&= - \frac{1}{2 \sigma ^2} \left( - 2 X^T Y + 2 X^T X \theta \right) \\
&= - \frac{1}{\sigma ^2} \left( X^T Y + X^T X \theta \right) \\
\end{aligned}
$$

Luego podemos usar toda nuestra artillería de optimización despejando $\partial l(\theta) / \partial \theta = 0$ y demostrando que es un máximo. Nuevamente llegamos a 

$$\hat{\theta} = (X^T X)^{-1} X^T Y$$

## Scikit-learn

_**[Scikit-learn](https://scikit-learn.org/)** Machine Learning in Python_

Scikit-learn is an open source machine learning library that supports supervised and unsupervised learning. It also provides various tools for model fitting, data preprocessing, model selection and evaluation, and many other utilities.

* Simple and efficient tools for predictive data analysis
* Accesible to everybody, and reusable in various contexts
* Built on `numpy`, `scipy` and `matplotlib`
* Open source, commercially usable - BSD license

Scikit-learn cuenta con enorme cantidad de herramientas de regresión, siendo la regresión lineal la más simple de estas. Ajustar una es tan sencillo como:

In [17]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression(fit_intercept=False)
reg.fit(X, Y)
theta_sklearn = reg.coef_.T
print(theta_sklearn)

[[200.]
 [ 50.]]


Nota que primero se crea un objeto `LinearRegression` en que se declaran algunos parámetros, por ejemplo, en nuestro caso la matriz de diseño `X` ya posee una columna de intercepto, por lo que no es necesario incluirla en el modelo de scikit-learn. Luego se ajusta el modelo `reg` con el método `fit()`.


### Benchmark

Implementación simple

In [18]:
%%timeit
lms_regression_slow(X, Y, theta_0)

1min 33s ± 1.2 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [19]:
theta_slow

array([[199.39672176],
       [ 50.19457286]])

Implementación vectorizada

In [20]:
%%timeit
lms_regression_fast(X, Y, theta_0)

241 ms ± 7.73 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [21]:
theta_fast

array([[199.39672176],
       [ 50.19457286]])

Implementación numpy

In [22]:
%%timeit
matrix_regression(X, Y, theta_0)

17.1 µs ± 475 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [23]:
theta_npsolve

array([[200.],
       [ 50.]])

Implementación scikit-learn

In [24]:
%%timeit
LinearRegression(fit_intercept=False).fit(X, Y).coef_.T

278 µs ± 2.58 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [25]:
theta_sklearn

array([[200.],
       [ 50.]])

Algunos comentarios:

- La implementación simple es **miles de veces** más lenta que la más rápida, que en este caso es la implementación de numpy.
- La implementación de numpy es sin duda la más rápida, pero no es posible utilizarla con matrices singulares.
- Las implementaciones de _gradient descent_ implementadas _from scratch_ no son lo sufiecientemente precisas.
- scikit-learn demora más pues es más flexible, además de realizar validaciones al momento de ajustar los modelos.

#### Aspectos Prácticos

Al realizar regresión, algunos autores indican que es conveniente normalizar/estandarizar los datos, es
decir transformarlos para que tengan una escala común:

-   Utilizando la media y la desviación estándar
    $$\frac{x_i-\overline{x_i}}{\sigma_{x_i}}$$

-   Utilizando mínimos y máximos
    $$\frac{x_i-\min{x_i}}{\max{x_i} - \min{x_i}}$$

**¿Porqué normalizar?**

-   Los valores numéricos poseen escalas de magnitud distintas.
-   Las variables tienen distintos significados físicos.
-   Algoritmos funcionan mejor.
-   Interpretación de resultados es más sencilla.

**Algunos problemas potenciales**
- Normalizar los datos puede producir colinealidad en los datos, produciendo inestabilidad numérica en la implementación.

En particular, scikit-learn ofrece herramientas para transformar datos. Por ejemplo, para escalar la forma más fácil y directa es utilizar `sklearn.preprocessing.scale`

In [26]:
from sklearn import preprocessing

diabetes_X_scaled = preprocessing.scale(diabetes_X)
diabetes_X_scaled

array([[ 0.80050009,  1.06548848,  1.29708846, ..., -0.05449919,
         0.41855058, -0.37098854],
       [-0.03956713, -0.93853666, -1.08218016, ..., -0.83030083,
        -1.43655059, -1.93847913],
       [ 1.79330681,  1.06548848,  0.93453324, ..., -0.05449919,
         0.06020733, -0.54515416],
       ...,
       [ 0.87686984,  1.06548848, -0.33441002, ..., -0.23293356,
        -0.98558469,  0.32567395],
       [-0.9560041 , -0.93853666,  0.82123474, ...,  0.55838411,
         0.93615545, -0.54515416],
       [-0.9560041 , -0.93853666, -1.53537419, ..., -0.83030083,
        -0.08871747,  0.06442552]])

Sin embargo, para parovechar todas las bondades de scikit-learn se recomienda hacer uso de los objetos `Transformer`.

In [27]:
scaler = preprocessing.StandardScaler().fit(diabetes_X)
scaler.transform(diabetes_X)

array([[ 0.80050009,  1.06548848,  1.29708846, ..., -0.05449919,
         0.41855058, -0.37098854],
       [-0.03956713, -0.93853666, -1.08218016, ..., -0.83030083,
        -1.43655059, -1.93847913],
       [ 1.79330681,  1.06548848,  0.93453324, ..., -0.05449919,
         0.06020733, -0.54515416],
       ...,
       [ 0.87686984,  1.06548848, -0.33441002, ..., -0.23293356,
        -0.98558469,  0.32567395],
       [-0.9560041 , -0.93853666,  0.82123474, ...,  0.55838411,
         0.93615545, -0.54515416],
       [-0.9560041 , -0.93853666, -1.53537419, ..., -0.83030083,
        -0.08871747,  0.06442552]])

## Aplicación

Para aplicar a nuestros datos de diabetes es tan fácil como 

In [28]:
reg = LinearRegression(fit_intercept=True).fit(diabetes_X, diabetes_y)

Para obtener los coeficientes de regresión y el intercepto debes acceder a los atributos de la instancia

In [29]:
reg.coef_

array([ -10.01219782, -239.81908937,  519.83978679,  324.39042769,
       -792.18416163,  476.74583782,  101.04457032,  177.06417623,
        751.27932109,   67.62538639])

In [30]:
reg.intercept_

152.1334841628965

También es posible predecir u obtener el score asociado a datos.

In [31]:
reg.predict(diabetes_X)

array([206.11706979,  68.07234761, 176.88406035, 166.91796559,
       128.45984241, 106.34908972,  73.89417947, 118.85378669,
       158.81033076, 213.58408893,  97.07853583,  95.1016223 ,
       115.06673301, 164.67605023, 103.07517946, 177.17236996,
       211.75953205, 182.84424343, 147.99987605, 124.01702527,
       120.33094632,  85.80377894, 113.11286302, 252.44934852,
       165.48821056, 147.72187623,  97.12824075, 179.09342974,
       129.05497324, 184.78138552, 158.71515746,  69.47588393,
       261.50255826, 112.81897436,  78.37194762,  87.66624129,
       207.92460213, 157.87686037, 240.84370686, 136.93372685,
       153.48187659,  74.15703284, 145.63105805,  77.8280105 ,
       221.0786645 , 125.22224022, 142.60147066, 109.4926324 ,
        73.14037106, 189.87368742, 157.93636782, 169.55816531,
       134.18186217, 157.72356219, 139.1077439 ,  72.73252701,
       207.8289973 ,  80.10834588, 104.08562488, 134.57807971,
       114.23779529, 180.67760064,  61.12644508,  98.72

In [32]:
reg.score(diabetes_X, diabetes_y)

0.5177494254132934